<div style="font-size: 200%; font-weight: bold; color: maroon;">The SIR model</div>
<!-- Paulo Villegas, 2021 - v. 1.1 -->

Example taken from [Learning Scientific Programming with Python](https://scipython.com/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

%matplotlib widget
plt.style.use('bmh')
plt.rcParams['figure.figsize'] = (8, 6)

# setup

In [ ]:
# Total population, N.
N = 1000

# Initial number of infected and recovered individuals, I0 and R0.
It0, Rt0 = 1, 0

# Everyone else, S0, is susceptible to infection initially.
St0 = N - It0 - Rt0

In [ ]:
# A grid of time points (in days)
t = np.linspace(0, 160, 160)

# Initial conditions vector
y0 = St0, It0, Rt0

The SIR model is governed by three differential equations, which give the variation of populations in compartments over time, $S(t)$, $I(t)$, $R(t)$:

$\frac{dS}{dt} = -\beta S \frac{I}{N}$

$\frac{dI}{dt} = \beta S \frac{I}{N} - \gamma I $

$\frac{dR}{dt} = \gamma I $

In [ ]:
# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dS_dt = -beta * S * I / N
    dI_dt = beta * S * I / N - gamma * I
    dR_dt = gamma * I
    return dS_dt, dI_dt, dR_dt

# process

The SIR model parameters are:
* $ \beta = $ effective contact rate
* $ \gamma = $ mean recovery rate

In [ ]:
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 0.4, 1./10 

In [ ]:
# Integrate the SIR equations over the time grid, t
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

# results

In [ ]:
# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')

ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Time /days')
ax.set_ylabel('Number (1000s)')
ax.set_ylim(0, 1.1)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)

fig.tight_layout()
fig.canvas.toolbar_position = 'bottom'

## dependence on $\beta$

In [ ]:
betalist = np.arange(0.1, 0.6, 0.05)

In [ ]:
gamma = 1./14 

In [ ]:
from collections import namedtuple
SIR = namedtuple('SIR', 'S I R')

def solve_sir(beta):
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    return SIR(*ret.T)

In [ ]:
result = [solve_sir(b) for b in betalist]

In [ ]:
%%html
<style>
div.widget-label {display: none;}
</style>

In [ ]:
from matplotlib.cm import get_cmap

cm = get_cmap('Wistia', len(betalist))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for n, (beta, sir) in enumerate(zip(betalist, result)):
    ax.plot(sir.I, label='{:.02f}'.format(beta), c=cm(n))
legend = ax.legend()
legend.get_frame().set_alpha(0.8)
ax.set_title(r'I(t) - dependence on $\beta$')
fig.canvas.toolbar_position = 'bottom'
fig.tight_layout()
fig.show()